In [77]:
import csv
import numpy as np
X = []
with open("./data/train/crawler/data/tweets.txt.text", newline='', encoding='utf8') as file_data:
    i = 0 
    for row in file_data:
        if i < 500:
            i = i +1
            X.append(row)
            
y = []
with open("./data/train/crawler/data/tweets.txt.labels", newline='', encoding='utf8') as file_data:
    j = 0
    for row in file_data:
        if j < 500:
            j = j +1
            y.append(row.replace("\n",""))


In [78]:
X = np.array(X)
y = np.array(y)

In [79]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

In [80]:
test=[]
with open("./data/test/us_test.text", newline='', encoding='utf8') as test_data:
    file = test_data.readlines()
    for row in file:
        test.append(row.replace("\n",""))

test = np.asarray(test)
test_dtm = vect.transform(test)

In [81]:
test_label = []
with open("./data/test/us_test.labels", newline='', encoding='utf8') as test_data_label:
    file = test_data_label.readlines()
    for row in file:
        test_label.append(row.replace("\n",""))
test_label = np.asarray(test_label)
test_label = test_label.reshape(-1,1)

In [106]:
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, f1_score, jaccard_score, classification_report


models = [
    LinearSVC()
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []

for model in models:
    model_name = model.__class__.__name__
    model.fit(X_dtm, y)
    predict_label = model.predict(test_dtm)
    acc = accuracy_score(predict_label,test_label)
    f1 = f1_score(predict_label, test_label, average = 'weighted')
    cm = multilabel_confusion_matrix(test_label,predict_label)
    jaccard = jaccard_score(test_label, predict_label, average='micro')
    print(classification_report(test_label, predict_label))
    print ("la matrice de confusion : ")
    print(cm)
    entries.append((model_name, acc, f1, jaccard))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'accuracy', 'f1', 'jaccard'])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

              precision    recall  f1-score   support

           0       0.26      0.40      0.32     10798
           1       0.13      0.11      0.12      4830
          10       0.07      0.04      0.06      1432
          11       0.17      0.10      0.12      1949
          12       0.13      0.14      0.14      1265
          13       0.03      0.01      0.02      1114
          14       0.05      0.02      0.03      1306
          15       0.05      0.03      0.03      1244
          16       0.03      0.02      0.02      1153
          17       0.57      0.46      0.51      1545
          18       0.11      0.03      0.05      2417
          19       0.05      0.00      0.01      1010
           2       0.16      0.24      0.19      4534
           3       0.08      0.08      0.08      2605
           4       0.23      0.19      0.21      3716
           5       0.05      0.06      0.06      1613
           6       0.05      0.04      0.04      1996
           7       0.11    

In [107]:
print (cv_df)

  model_name  accuracy        f1   jaccard
0  LinearSVC   0.17384  0.190368  0.095194


In [34]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

parameters = {'multi_class':('crammer_singer', 'ovr'), 'class_weight':['balanced'],
             'loss':('hinge', 'squared_hinge'), 'penalty':['l2'], 'C':[0.5, 1, 2]}
lsvc = LinearSVC()

clf = GridSearchCV(lsvc, parameters, cv=3, n_jobs=-1)

clf.fit(X_dtm, y)

print(clf.cv_results_)

{'mean_fit_time': array([0.05892825, 0.02644642, 0.05153545, 0.04271213, 0.07403366,
       0.04101721, 0.05581911, 0.04363139, 0.04777376, 0.02871299,
       0.03998351, 0.04367193]), 'std_fit_time': array([0.00757584, 0.00473171, 0.01752554, 0.01689881, 0.00763784,
       0.00535953, 0.01536049, 0.0046304 , 0.00903355, 0.00320662,
       0.01132276, 0.01754775]), 'mean_score_time': array([0.00137639, 0.00084662, 0.00075157, 0.00077391, 0.00100358,
       0.00301449, 0.00077049, 0.00079536, 0.00071295, 0.00071677,
       0.0006752 , 0.00063173]), 'std_score_time': array([2.28741059e-04, 7.73542457e-05, 1.91422389e-05, 8.05730171e-05,
       2.98515189e-04, 3.24723088e-03, 6.23102698e-05, 8.21021186e-05,
       1.12543222e-05, 3.31515202e-05, 5.16849835e-05, 8.56305220e-05]), 'param_C': masked_array(data=[0.5, 0.5, 0.5, 0.5, 1, 1, 1, 1, 2, 2, 2, 2],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False],
       fill_val

/home/julien/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
